In [1]:
import pandas as pd
import numpy as np
import string
from collections import Counter
import random

In [2]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# to break into sentences from text
def sentence(text):
    sentences = []
    sentences = list(text.split("\n"))
    return sentences
    

# load the document
filename = 'a1_data/a1_d3.txt'
text = load_doc(filename)
sentences = sentence(text)

In [3]:
# text preprocessing
lower_case_sentences = []
for i in sentences:
    lower_case_sentences.append(i.lower())

no_punctuations = []
for i in lower_case_sentences:
    no_punctuations.append(''.join(c for c in i if c not in string.punctuation))

clean_data = []
for i in no_punctuations:
    sub = i.split(', ')
    sub1 = sub[0].split('\t')
    clean_data.append(sub1)
clean_data.remove(clean_data[-1])

In [4]:
df = pd.DataFrame(clean_data, columns =['Review', 'Sentiment'])
print(df)

                                                Review Sentiment
0    so there is no way for me to plug it in here i...         0
1                            good case excellent value         1
2                                great for the jawbone         1
3    tied to charger for conversations lasting more...         0
4                                     the mic is great         1
..                                                 ...       ...
995  the screen does get smudged easily because it ...         0
996  what a piece of junk i lose more calls on this...         0
997                        item does not match picture         0
998  the only thing that disappoint me is the infra...         0
999  you can not answer calls with the unit never w...         0

[1000 rows x 2 columns]


In [5]:
# Split a dataset into 5 folds
def cross_validation(df, n_folds):
    df_split = list()
    df_copy = list(df)
    fold_size = int(len(df) / 5)
    for i in range (n_folds):
        fold = []
        while len(fold) < fold_size:
            index = random.randrange(0,len(df_copy))
            fold.append(df_copy.pop(index))
        df_split.append(fold)
    return df_split

folds = cross_validation(clean_data, 5)
for fold in folds:
    train_set = list(folds)
    train_set.remove(fold)
    train_set = sum(train_set, [])
    test_set = list()
    for row in fold:
        row_copy = list(row)
        test_set.append(row_copy)
        
train_df = pd.DataFrame(train_set, columns =['Review', 'Sentiment'])
test_df = pd.DataFrame(test_set,columns =['Review', 'Sentiment'])
train_df_positive = train_df.loc[train_df['Sentiment']=='1']
train_df_negative = train_df.loc[train_df['Sentiment']=='0']

# Setting the model's vocabulary
def vocab_freq(train_df):
    train_sentences = train_df['Review'].values
    train_sentences_list = train_sentences.tolist()
    all_words_train = []
    for i in train_sentences_list:
        all_words_train.extend(i.split(' '))
    vocab,count = np.unique(np.array(all_words_train),return_counts=True)
    return (vocab,count)

#Setting the positive sentiment and negative sentiment vocab and frequency
vocab_positive, count_positive = vocab_freq(train_df_positive)
vocab_negative, count_negative = vocab_freq(train_df_negative)
vocab_total, count_total = vocab_freq(train_df)


#Gives the probability P(C) or prior probability
# no. of sentiment values is the same as the no. of reviews in train_set
train_sentiments = train_df['Sentiment'].values
sentiment,count = np.unique(train_sentiments,return_counts=True)

positive_review_count = count[1]
negative_review_count = count[0]

prob_positive = positive_review_count / (positive_review_count + negative_review_count)
prob_negative = negative_review_count / (positive_review_count + negative_review_count)

In [7]:
# extracting the words from the test_set
test_sentences = test_df['Review'].values
test_sentences_list = test_sentences.tolist()
words_test = []
for i in test_sentences_list:
    words_test.append(i.split(' '))
print (list(words_test).index)

#Calculating likelihood probability P(d|C)
#Writing a function for a given class C = 1,0
def posterior_prob(train_df,vocab,count,words_test,prob_class, class_count):
    posterior_prob = list()
    #Calculations for test data in row i
    for i in words_test:
        likelihood_prob = 1
        word_test_array = np.array(i)
        vocab_test,count_test = np.unique(word_test_array,return_counts=True)
        #j returns the elements of the iterable list i
        for j in i:
            try:
                index = list(vocab).index(j)
            except ValueError:
                continue
            index1 = list(vocab_test).index(j)
            #here likelihood probability is returned for the ith row of test data
            likelihood_prob *= count_test[index1]*((count[index]+1)/(np.sum(count)+class_count))
        #return the probability P(d|C)*P(C)
        posterior = prob_class*likelihood_prob
        posterior_prob.append(posterior)
    return posterior_prob
            
posterior_prob_positive = posterior_prob(train_df,vocab_positive,count_positive,words_test,prob_positive,positive_review_count)
posterior_prob_negative = posterior_prob(train_df,vocab_negative,count_negative,words_test,prob_negative,negative_review_count)
print (posterior_prob_positive)

<built-in method index of list object at 0x7f58936de820>
[2.4395721745304103e-09, 2.4295171532861695e-41, 5.74837690927997e-13, 2.6138273298540106e-10, 2.3530906773550855e-16, 1.4041638861405757e-13, 0.0002317497103128621, 3.8315942210391855e-49, 4.1200920593660167e-38, 2.283915237741065e-40, 7.58650293946324e-37, 6.916534345166263e-16, 3.373787529145074e-32, 2.5898543829638684e-11, 1.3776319557691678e-33, 5.269733697856123e-13, 1.8973975542985289e-10, 6.454434791532202e-16, 6.665259691127728e-08, 1.386636081037784e-15, 5.0177124431447315e-42, 1.726569588642579e-12, 1.5360467473807316e-05, 1.826069559823247e-05, 2.7186123963605865e-35, 1.3199828015762753e-08, 0.00220162224797219, 3.187020665476371e-12, 9.581680823249732e-49, 2.888279199488682e-07, 4.895073185980047e-46, 4.5920278636731656e-05, 2.1912834717878967e-05, 5.754184188593717e-14, 1.350194873972976e-10, 5.478208679469742e-06, 5.084278483477002e-41, 1.996470730195888e-33, 0.0008111239860950173, 2.041274676647894e-09, 9.99147861

In [8]:
# implementing f1 score
def f1_score_single(y_actual, y_pred):
    y_actual = set(y_actual)
    y_pred = set(y_pred)
    true_positives = len(y_actual & y_pred)
    if true_positives == 0: return 0.
    precision = 1.0* true_positives / len(y_pred)
    recall = 1.0* true_positives / len(y_actual)
    return 2 * precision * recall / (precision + recall)
    
def f1_score(y_actual, y_pred):
    return np.mean([f1_score_single(x, y) for x, y in zip(y_actual, y_pred)])

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [9]:
test_predict = list()

#predict the Sentiment
for i in range (len(test_set)):
    if posterior_prob_positive[i] > posterior_prob_negative[i]:
        test_predict.append("1")
    else:
        test_predict.append("0")
        
test_df['Predicted Sentiment'] = test_predict 
f1_score = f1_score(test_df['Sentiment'], test_df['Predicted Sentiment'])
accuracy = accuracy_metric(test_df['Sentiment'], test_df['Predicted Sentiment'])
print(test_df)
print(f1_score)
print(accuracy)

                                                Review Sentiment  \
0                           having trouble with volume         0   
1    this is cool because most cases are just open ...         1   
2             the construction of the headsets is poor         0   
3                                     doesnt last long         0   
4                           will order from them again         1   
..                                                 ...       ...   
195          we have gotten a lot of compliments on it         1   
196                                       fast service         1   
197                 excellent starter wireless headset         0   
198                 plantronics bluetooth excelent buy         1   
199                           good product good seller         1   

    Predicted Sentiment  
0                     1  
1                     0  
2                     1  
3                     0  
4                     0  
..                  ...  
1